In the notebook, we demonstrate the capabilities of the Robot class. This class provides an easy interface for Casadi functions for robot kinematics and dynamics. 

These CasADi functions are obained using the state-of-the-art rigid body dynamics library Pinocchio.

- Original URDF2CASADI Robot module name is 'robotsmeco' from MECO Team, KU Leuven.
- Please refer to https://gitlab.kuleuven.be/meco-software/robot-models-meco
```
from robotsmeco import Robot as rob
```
- In this example, customized robot module is adopted: https://github.com/SHKim-HYU/robot-model-hyu
```
from robotshyu import Robot as rob
```

In [1]:
# Customized module from Biorobotics LAB, HYU
from robotshyu import Robot as rob

Load the desired robot as an instance of the Robot class. The possible options currently supported for the robot are:
"atlas",  "franka_panda", "iiwa14", "kinova",  "kr60",  "yumi", "franka_panda_with_gripper",  "iiwa7",   "kinova_gripper",  "kr5",  "ur10", "indy7", "indyrp2", "mmo_500_ppr", "m0609", "xarm6"

In [2]:
robot = rob.Robot("indy7", analytical_derivatives=True)

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [3]:
robot.fk

Function(fk_T:(q[6])->(T_joint0[4x4],T_joint1[4x4],T_joint2[4x4],T_joint3[4x4],T_joint4[4x4],T_joint5[4x4],T_indy7_tcp[4x4]) SXFunction)

The robot fk function takes as input the joint angles and provides as output the Casadi expressions for the pose of all the links present in the URDF file.

In [4]:
robot.fk([0]*robot.ndof)[6]

DM(
[[1, -4.44089e-16, 1.47911e-31, -1.6398e-16], 
 [4.44089e-16, 1, -4.44089e-16, -0.1865], 
 [4.93038e-32, 4.44089e-16, 1, 1.3275], 
 [0, 0, 0, 1]])

Robot object contains default position and velocity limits of the robot joint positions and velocities, that is read from the URDF file

In [5]:
print("Lower bounds of the robot joint positions: ", robot.joint_lb)
print("Upper bounds of the robot joint positions: ", robot.joint_ub, "\n\n")

Lower bounds of the robot joint positions:  [-3.05433, -3.05433, -3.05433, -3.05433, -3.05433, -3.75246]
Upper bounds of the robot joint positions:  [3.05433, 3.05433, 3.05433, 3.05433, 3.05433, 3.75246] 




The robot jacobian(space, body) can be calculated using J_s, J_b function

In [6]:
robot.J_s([0]*robot.ndof)

DM(
[[0, 0.2995, 0.7495, -0.0035, 1.0995, -0.1865], 
 [0, 6.65024e-17, 1.66422e-16, 1.82632e-16, 4.88276e-16, 1.6398e-16], 
 [0, 2.42029e-17, 1.24123e-16, 4.07249e-32, 2.09499e-16, 1.00407e-31], 
 [0, 2.22045e-16, 2.22045e-16, 4.93038e-32, 4.44089e-16, 1.47911e-31], 
 [0, -1, -1, -2.22045e-16, -1, -4.44089e-16], 
 [1, 0, 0, 1, -2.22045e-16, 1]])

In [7]:
robot.J_b([0]*robot.ndof)

DM(
[[0.1865, -1.028, -0.578, 0.183, -0.228, 0], 
 [-2.46803e-16, 2.28262e-16, 1.28342e-16, -6.26166e-17, -1.2326e-32, 0], 
 [1.00407e-31, -1.81188e-16, -8.12683e-17, 1.39037e-32, -3.73035e-17, 0], 
 [4.93038e-32, -2.22045e-16, -2.22045e-16, 2.18953e-47, 0, 4.37906e-47], 
 [4.44089e-16, -1, -1, 2.22045e-16, -1, 9.86076e-32], 
 [1, 4.44089e-16, 4.44089e-16, 1, 2.22045e-16, 1]])

Efficient inverse dynamics and forward dynamics expressions are also available.

In [8]:
robot.id

Function(id:(q[6],q_dot[6],q_ddot[6])->(tau[6]) SXFunction)

In [9]:
# arg: q, qdot, qddot
robot.id([0]*robot.ndof,[0]*robot.ndof,[0]*robot.ndof)

DM([-9.86076e-32, -1.05351e-14, -2.88546e-15, -1.14582e-31, -1.46686e-16, 0])

In [10]:
robot.fd

Function(fd:(q[6],q_dot[6],tau[6])->(q_ddot[6]) SXFunction)

The forward dynamics functions takes joint angles, velocities and joint torques as input and returns the joint accelerations.

In [11]:
# arg: q, qdot, tau
robot.fd([0]*robot.ndof, [0]*robot.ndof, [0]*robot.ndof)

DM([1.25131e-15, 5.62301e-15, -3.40275e-15, 1.99883e-14, 1.06352e-14, -2.12286e-14])

Dynamics function such as Mass matrix, Mass inverse, Coriolis, Gravity

In [12]:
# arg: q
robot.M([0]*robot.ndof)

DM(
[[0.563162, -0.39724, -0.13356, 0.0610756, -0.0125628, 0.00059634], 
 [-0.39724, 3.58424, 1.25873, -0.278455, 0.0683465, -5.1e-07], 
 [-0.13356, 1.25873, 0.549731, -0.128038, 0.038043, -5.1e-07], 
 [0.0610756, -0.278455, -0.128038, 0.0599057, -0.0123271, 0.00059634], 
 [-0.0125628, 0.0683465, 0.038043, -0.0123271, 0.0144737, -5.1e-07], 
 [0.00059634, -5.1e-07, -5.1e-07, 0.00059634, -5.1e-07, 0.00059634]])

In [13]:
# arg: q
robot.Minv([0]*robot.ndof)

DM(
[[2.06565, 0.326748, -0.782851, -2.16618, 0.46274, 0.10054], 
 [0.326748, 1.54557, -3.81111, -0.826595, 2.2985, 0.499875], 
 [-0.782851, -3.81111, 13.2095, 9.48393, -9.32623, -8.70101], 
 [-2.16618, -0.826595, 9.48393, 37.5488, 9.07401, -35.3674], 
 [0.46274, 2.2985, -9.32623, 9.07401, 90.8799, -9.46504], 
 [0.10054, 0.499875, -8.70101, -35.3674, -9.46504, 1712.15]])

In [14]:
# arg: q, qdot
robot.C([0]*robot.ndof,[0]*robot.ndof)

DM(
[[0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0]])

In [15]:
# arg: q
robot.G([0]*robot.ndof)

DM([-9.86076e-32, -1.05351e-14, -2.88546e-15, -1.14582e-31, -1.46686e-16, 0])

Analytical derivatives of robot dynamics computed using Pinocchio is also available. The analytical derivative computation can be up to two times faster than automatic differentiation. 

In [16]:
robot.J_fd

Function(jac_fd:(q[6],q_dot[6],tau[6])->(jac_fd[6x18]) SXFunction)

This submodule of Tasho is almost a stand-alone contribution. These CasADi expressions can be used by anyone that is implementing robot MPC/OCP using CasADi, even if they are not using Tasho.